In [1]:
import requests
import xarray as xr
import matplotlib.pyplot as plt
import cmocean
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from datetime import datetime, timedelta
import geopandas
from shapely.geometry import box
import netCDF4 as nc

/Users/alonso.gonzalez.glez/anaconda3/envs/geocoral/lib/python3.11/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.9.1dev-CAPI-1.14.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.4-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


In [3]:
import os
desktop = os.path.join(os.path.expanduser('~'), 'Desktop')
folder = 'GeoCoral'
new_folder_path = os.path.join(desktop, folder)
if not os.path.exists(new_folder_path):
    os.makedirs(new_folder_path)
    print(f"Folder '{folder}' created successfully at {desktop}")
else:
    print(f"Folder '{folder}' already exists at {desktop}")

Folder 'GeoCoral' created successfully at /Users/alonso.gonzalez.glez/Desktop


In [4]:
import gdown
import geopandas as gpd
import zipfile
import os

def download_file_from_google_drive(file_id, dest_path):
    url = f'https://drive.google.com/uc?id={file_id}'
    gdown.download(url, dest_path, quiet=False)

def extract_zip(zip_path, extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)

file_id = '1LNiRXvsiWqoUuGHSQduWBFjfsuy83Efs'
zip_destination = 'Corals.zip'
download_file_from_google_drive(file_id, zip_destination)
extraction_folder = new_folder_path
if not os.path.exists(extraction_folder):
    os.makedirs(extraction_folder)
extract_zip(zip_destination, extraction_folder)


Downloading...
From: https://drive.google.com/uc?id=1LNiRXvsiWqoUuGHSQduWBFjfsuy83Efs
To: /Users/alonso.gonzalez.glez/Documents/Repositories/Geo_Coral/etl/Corals.zip
100%|██████████| 32.6k/32.6k [00:00<00:00, 9.88MB/s]


In [5]:
import requests
from datetime import datetime, timedelta
#20,-24,89,172
current_date = datetime.now()
two = current_date - timedelta(days=6)
date_two = two.strftime('%Y-%m-%d')
link= "https://coastwatch.pfeg.noaa.gov/erddap/griddap/NOAA_DHW.nc?CRW_DHW%5B("+date_two+"T12:00:00Z):1:(2024-02-15T12:00:00Z)%5D%5B(-24.0):1:(20.0)%5D%5B(89.0):1:(172.0)%5D"
response = requests.get(link)
print(response)

<Response [200]>


In [ ]:
#Opening NetCDF
dataset = xr.open_dataset(response.content)
sst = dataset['CRW_DHW'][0].values
lats = dataset['latitude'].values
lons = dataset['longitude'].values

# Create a Cartopy plot
plt.figure(figsize=(10, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.add_feature(cfeature.COASTLINE)
ax.add_feature(cfeature.BORDERS, linestyle=':')
ax.gridlines(draw_labels=True)
#Plotting DHW
cax = plt.pcolormesh(lons, lats, sst, shading='auto', cmap=cmocean.cm.thermal, transform=ccrs.PlateCarree())
plt.colorbar(cax, label='Degree Heat Week', orientation='horizontal', pad=0.06)
        
plt.title('Degree Heat Week')


In [ ]:
min_value = dataset['latitude'].values
height, width = sst.shape


In [8]:
import geopandas as gpd
import rasterio
from rasterio.transform import from_origin
from rasterio.enums import Resampling
from rasterio.warp import calculate_default_transform, reproject, Resampling

# Replace this with the actual bounding box you want
target_bbox = {
    'minx': 89,
    'miny': 64,
    'maxx': 172,
    'maxy': 20
}

tiff_name = new_folder_path+"/DHW_"+date_two+".tif"
print(tiff_name)
height, width = sst.shape

transform = from_origin(target_bbox['minx'], target_bbox['maxy'],
                        (target_bbox['maxx'] - target_bbox['minx']) / width,
                        (target_bbox['miny'] - target_bbox['maxy']) / height)

with rasterio.open(tiff_name, 'w', driver='GTiff', height=height, width=width, count=1,
                   dtype=str(sst.dtype), crs=4326, transform=transform) as dst:
    dst.write(sst, 1)

/Users/alonso.gonzalez.glez/Desktop/GeoCoral/DHW_2024-02-17.tif


In [9]:
import psycopg2


new_connection = psycopg2.connect(user = "postgres", 
                                  password = "postgres", host = "localhost", port = "5432")
new_connection.autocommit = True
cursor = new_connection.cursor()

query = "CREATE DATABASE geocoral_c"
cursor.execute(query)

In [10]:
import psycopg2


new_connection = psycopg2.connect(database='geocoral_c' ,user = "postgres", 
                                  password = "postgres", host = "localhost", port = "5432")
new_connection.autocommit = True
cursor = new_connection.cursor()


In [12]:
table_query = """
CREATE TABLE IF NOT EXISTS corals (
    id SERIAL PRIMARY KEY,
    geometry GEOMETRY(Polygon, 4326) 
);
"""

new_connection.autocommit = True
cursor = new_connection.cursor()
cursor.execute(table_query)

In [13]:
import geopandas as gpd

shapefile_path = new_folder_path + '/Corals.geojson'
print(shapefile_path)
gdf = gpd.read_file(shapefile_path)
print(gdf)

/Users/alonso.gonzalez.glez/Desktop/GeoCoral/Corals.geojson
      ID                                           geometry
0      0  POLYGON ((131.02729 -8.21997, 131.02729 -8.224...
1      1  POLYGON ((131.09017 -8.19257, 131.09017 -8.188...
2      2  POLYGON ((131.12161 -8.17430, 131.12161 -8.178...
3      3  POLYGON ((130.81618 -8.16060, 130.81618 -8.165...
4      4  POLYGON ((131.12161 -8.17430, 131.12161 -8.169...
..   ...                                                ...
225  225  POLYGON ((131.57077 -6.72440, 131.56628 -6.724...
226  226  POLYGON ((131.58874 -6.71985, 131.58424 -6.719...
227  227  POLYGON ((131.57077 -6.71530, 131.57077 -6.710...
228  228  POLYGON ((131.53035 -6.73806, 131.53035 -6.733...
229  229  POLYGON ((131.59323 -6.66067, 131.59323 -6.656...

[230 rows x 2 columns]


In [14]:
from shapely.wkt import dumps
for index, row in gdf.iterrows():
    geom_wkt = dumps(row['geometry'])
    print(geom_wkt)
    print(row)
    insert_query = (
        "INSERT INTO corals (ID, geometry) VALUES (%s, ST_GeomFromText(%s, 4326))"
    )
    cursor.execute(insert_query, (row['ID'], geom_wkt))
    

# Commit changes and close the connection
new_connection.commit()
cursor.close()
new_connection.close()

POLYGON ((131.0272889999067445 -8.2199717543584168, 131.0272889999067445 -8.2245393146453818, 131.0227974234861676 -8.2245393146453818, 131.0227974234861676 -8.2291069261821388, 131.0138142706450139 -8.2291069261821388, 131.0138142706450139 -8.2336745889986833, 131.0093226942243518 -8.2336745889986833, 131.0093226942243518 -8.2382423031251477, 131.0048311178037750 -8.2382423031251477, 131.0048311178037750 -8.2291069261821388, 131.0003395413831697 -8.2291069261821388, 131.0003395413831697 -8.2336745889986833, 130.9913563885419592 -8.2336745889986833, 130.9913563885419592 -8.2291069261821388, 130.9958479649625644 -8.2291069261821388, 130.9958479649625644 -8.2245393146453818, 131.0048311178037750 -8.2245393146453818, 131.0048311178037750 -8.2199717543584168, 131.0138142706450139 -8.2199717543584168, 131.0138142706450139 -8.2154042452912126, 131.0183058470655624 -8.2154042452912126, 131.0183058470655624 -8.2199717543584168, 131.0272889999067445 -8.2199717543584168))
ID                     

In [22]:
import fiona
import rasterio
import rasterio.mask
from shapely.geometry import shape, mapping
import geopandas as gpd
import numpy as np
import pandas as pd

buffers_path = new_folder_path + '/Corals.geojson'

# Buffer function
def buffer_polygon(geometry, buffer_distance):
    polygon = shape(geometry)
    buffered_polygon = polygon.buffer(buffer_distance)
    return mapping(buffered_polygon)

# Buffer distance in degrees
buffer_distance = 0.025

with rasterio.open(tiff_name) as src:
    # Create an empty list to store the results
    results = []

    for feat in fiona.open(buffers_path):
        feature_id = feat['id']
        original_geometry = feat['geometry']

        # Buffer the polygon
        buffered_geometry = buffer_polygon(original_geometry, buffer_distance)

        # Masking
        out_image, out_transform = rasterio.mask.mask(src, [buffered_geometry], crop=True)
        average_value = np.nanmax(out_image)

        # Append the results to the list
        results.append({'ID': feature_id, 'geometry': original_geometry, 'buffered_geometry': buffered_geometry, 'average_value': average_value, 'Date': date_two})

# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(results)

# Drop the original geometry column if you don't need it
df = df.drop('buffered_geometry' ,axis=1)
df = df.drop('geometry' ,axis=1)

print(df)


      ID  average_value        Date
0      0           2.94  2024-02-17
1      1           1.88  2024-02-17
2      2           1.88  2024-02-17
3      3           2.87  2024-02-17
4      4           1.73  2024-02-17
..   ...            ...         ...
225  225           2.89  2024-02-17
226  226           3.30  2024-02-17
227  227           0.00  2024-02-17
228  228           3.45  2024-02-17
229  229           3.45  2024-02-17

[230 rows x 3 columns]


In [ ]:
import psycopg2
import pandas as pd

# Connect to the database
new_connection_b = psycopg2.connect(
    database="geocoral_c",
    user="postgres",
    password="postgres",
    host="localhost",
    port="5432"
)
new_connection_b.autocommit = True
cursor_b = new_connection_b.cursor()

# Create the table if it doesn't exist
table_query = """
CREATE TABLE IF NOT EXISTS measurements (
    id INTEGER,
    date DATE,
    average_value DOUBLE PRECISION  -- Change to the appropriate data type for average_value
);
"""
cursor_b.execute(table_query)

# Assuming df is your DataFrame with 'ID', 'DateFormatted', and 'average_value' columns
for index, row in df.iterrows():
    id = row['ID']
    date_formatted = row['Date']
    average_value = row['average_value']

    insert_query = "INSERT INTO measurements (date, average_value, id) VALUES (%s, %s, %s)"
    cursor_b.execute(insert_query, (date_formatted, average_value, id))

# Commit changes and close the connection
new_connection_b.commit()
cursor_b.close()
new_connection_b.close()

In [ ]:



{
  const myLatLng = { -1826138.68364, 4828977.43109, 8099634.76175, 14251717.88392 };

  map = new google.maps.Map(document.getElementById("map"), {
center: myLatLng,
zoom: 8,
projection: "EPSG:32549" 
}

initMap();